In [7]:
from langchain_community.llms import Ollama

# Specify the remote server's URL
llm = Ollama(model="phi4:latest", base_url="http://127.0.0.1:11434")

In [8]:
from langchain.chains import LLMMathChain

llm_math = LLMMathChain.from_llm(llm, verbose=True)

In [9]:

llm_math.prompt.template

'Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\n```text\n37593**(1/5)\n```\n...numexpr.evaluate("37593**(1/5)")...\n```output\n8.222831614237718\n```\nAnswer: 8.222831614237718\n\nQuestion: {question}\n'

In [10]:
expression = "551368 / 82"
expression = expression.strip()


In [11]:
print("Evaluating expression:", expression)


Evaluating expression: 551368 / 82


In [12]:
import numexpr

try:
    result = numexpr.evaluate(expression)
except ValueError as e:
    print("Error evaluating expression:", e)
    # Further debugging or input cleaning here


In [13]:
i = 0
example_query = "What is 551368 divided by 82"
llm_math.invoke(example_query)



> Entering new LLMMathChain chain...
What is 551368 divided by 82To solve the problem of dividing 551368 by 82 using Python's numexpr library, you would create and evaluate an expression as follows:

```text
551368 / 82
```

Using numexpr to evaluate this expression:

...numexpr.evaluate("551368 / 82")...

The output from running this code is:



ValueError: unknown format from LLM: To solve the problem of dividing 551368 by 82 using Python's numexpr library, you would create and evaluate an expression as follows:

```text
551368 / 82
```

Using numexpr to evaluate this expression:

...numexpr.evaluate("551368 / 82")...

The output from running this code is:

In [5]:
llm_math.invoke({"question": "What is 551368 divided by 82?"})




> Entering new LLMMathChain chain...
What is 551368 divided by 82?To solve the problem of dividing 551368 by 82 using Python's numexpr library, we can translate this into a single line mathematical expression and execute it.

```text
551368 / 82
```

...numexpr.evaluate("551368 / 82")...



ValueError: unknown format from LLM: To solve the problem of dividing 551368 by 82 using Python's numexpr library, we can translate this into a single line mathematical expression and execute it.

```text
551368 / 82
```

...numexpr.evaluate("551368 / 82")...

In [7]:
import numexpr

result = numexpr.evaluate("551368 / 82")
print(result)  # Should output: [6724]


6724.0


### Change the prompt to solve the above error in mathchain

In [14]:
from langchain.chains import LLMMathChain
from langchain.llms import Ollama
from langchain.prompts import PromptTemplate

# Create a custom prompt template to enforce the expected output.
custom_prompt = PromptTemplate(
    input_variables=["question"],
    template=(
        "You are a expert math solver. Solve the problem below and output only the final answer in the exact format:\n"
        "Answer: <result>\n\n"
        "Do not include any explanation, commentary, or extra text.\n\n"
        "Round the result with 2. For example,if the result is 5.6666666, make the result 5.66.\n\n"
        "Problem: {question}"
    )
)


# custom_prompt = PromptTemplate(
#     input_variables=["question"],
#     template=(
#         "You are an expert math solver. Solve the following problem using exact arithmetic. "
#         "Only output the final answer in the exact format 'Answer: <result>' with no additional text.\n\n"
#         "Problem: {question}"
#     )
# )




# Use your local Ollama model.
llm = Ollama(model="phi4:latest", base_url="http://127.0.0.1:11434", temperature = 0)

# Instantiate the math chain with the custom prompt.
math_chain = LLMMathChain(llm=llm, prompt=custom_prompt, verbose = True)

# Run the chain with the math problem.
# result = math_chain.invoke("What is 551368 divided by 82?")
result = math_chain.invoke("What is 551121 divided by 82?")
# result = math_chain.invoke("What is (12 plus 18 minus 10) devide by 3?")
print(result)  # Expected output: Answer: 6724


/tmp/ipykernel_5349/1055621017.py:34: LangChainDeprecationWarning: This class is deprecated and will be removed in langchain 1.0. See API reference for replacement: https://api.python.langchain.com/en/latest/chains/langchain.chains.llm_math.base.LLMMathChain.html
  math_chain = LLMMathChain(llm=llm, prompt=custom_prompt, verbose = True)
/home/mahdi/Math_LangChain/.venv/lib/python3.12/site-packages/langchain/chains/llm_math/base.py:175: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(




> Entering new LLMMathChain chain...
What is 551121 divided by 82?Answer: 6727.12
> Finished chain.
{'question': 'What is 551121 divided by 82?', 'answer': 'Answer: 6727.12'}


In [15]:
import re

# Assume result is the string returned by math_chain.run(...)
# Example: result = "Answer: 6724.731707317073"
if "Answer:" in result:
    # Extract the numeric part.
    numeric_part = result.split("Answer:")[-1].strip()
    try:
        # Convert to float and check if it's nearly an integer.
        value = float(numeric_part)
        if abs(value - round(value)) < 1e-6:
            value = round(value)
        # Format the final answer.
        final_answer = f"Answer: {value}"
    except Exception as e:
        final_answer = result
else:
    final_answer = result

print(final_answer)  # Expected output: Answer: 6724


{'question': 'What is 551121 divided by 82?', 'answer': 'Answer: 6727.12'}


In [16]:
# Direct computation in Python:
expression = "551368 / 82"
# Using Python's exact division (or integer division if appropriate):
result_value = eval(expression)
# If you expect an integer result:
if result_value == int(result_value):
    result_value = int(result_value)
print(f"Answer: {result_value}")  # Expected output: Answer: 6724


Answer: 6724


### Because the result is not accurate, I will try to use a python code to calculate mathematics part.

In [15]:
import re

def prompt_to_expression(prompt):
    # Convert prompt to lowercase to standardize the mapping.
    text = prompt.lower()
    # Map common arithmetic phrases to their operators.
    text = text.replace("divided by", "/")
    text = text.replace("multiplied by", "*")
    text = text.replace("plus", "+")
    text = text.replace("minus", "-")
    
    # Remove unnecessary words and punctuation.
    text = re.sub(r"what is", "", text)
    text = re.sub(r"\?", "", text)
    
    # Clean up extra spaces.
    return text.strip()

# Example prompt:
# prompt = "What is 551368 divided by 82?"
# prompt = "What is 2 multiplied by 82 minus 12.5?"
prompt = "What is 2 multiplied by 82 minus 12.5 plus 2?"
# prompt = "I want to 2 multiplied by 82. How much is it?"
expression = prompt_to_expression(prompt)
print("Expression:", expression)  # Expected output: "551368 / 82"

# Now evaluate the expression.
result_value = eval(expression)

# If the result is an integer, cast it accordingly.
if result_value == int(result_value):
    result_value = int(result_value)

print(f"Answer: {result_value}")  # Expected output: Answer: 6724


Expression: 2 * 82 - 12.5 + 2
Answer: 153.5


In [4]:
import re

def prompt_to_expression(prompt: str) -> str:
    """
    Convert a natural language math prompt into an arithmetic expression.
    
    This function uses simple heuristics:
      - It maps key phrases to arithmetic operators.
      - It extracts numbers from the prompt.
      - It then constructs an expression using the first two numbers found.
    
    Args:
        prompt (str): The natural language math prompt.
    
    Returns:
        str: A string representing the arithmetic expression.
    
    Raises:
        ValueError: If no valid operation or not enough numbers are found.
    """
    # Convert prompt to lowercase for uniform processing.
    text = prompt.lower()

    # Mapping of keywords/phrases to arithmetic operators.
    operations = {
        'divided by': '/',
        'divide': '/',
        'multiplied by': '*',
        'multiply': '*',
        'times': '*',
        'plus': '+',
        'added to': '+',
        'minus': '-',
        'subtracted by': '-',
        'subtract': '-',
        'power' : '^'
    }
    
    # Identify the operator by looking for any of the defined phrases.
    operator = None
    for phrase, op in operations.items():
        if phrase in text:
            operator = op
            break

    if operator is None:
        raise ValueError("No valid arithmetic operation found in the prompt.")

    # Extract numbers (both integers and decimals) from the prompt.
    numbers = re.findall(r'\d+(?:\.\d+)?', text)
    if len(numbers) < 2:
        raise ValueError("Not enough numbers found in the prompt to form an expression.")

    # Construct the arithmetic expression using the first two numbers.
    expression = f"{numbers[0]} {operator} {numbers[1]}"
    return expression



In [ ]:
# Example usage:
prompts = [
    "What is 551368 divided by 82?",
    "I want to multiply 2 and 82",
    "Please add 10.5 plus 2.3",
    "Subtract 20 from 100",
    "I want to multiply 2 and 12 minus 10 power 2"
]

for prompt in prompts:
    try:
        expression = prompt_to_expression(prompt)
        result_value = eval(expression)
        # If the result is a float that is effectively an integer, cast it.
        if result_value == int(result_value):
            result_value = int(result_value)
        print(f"Prompt: {prompt}")
        print(f"Extracted Expression: {expression}")
        print(f"Answer: {result_value}\n")
    except ValueError as e:
        print(f"Error processing prompt: {prompt}\n{e}\n")


Prompt: What is 551368 divided by 82?
Extracted Expression: 551368 / 82
Answer: 6724

Prompt: I want to multiply 2 and 82
Extracted Expression: 2 * 82
Answer: 164

Prompt: Please add 10.5 plus 2.3
Extracted Expression: 10.5 + 2.3
Answer: 12.8

Prompt: Subtract 20 from 100
Extracted Expression: 20 - 100
Answer: -80

Prompt: I want to multiply 2 and 12 minus 10
Extracted Expression: 2 * 12
Answer: 24



#### above solution is failed to express math equation with more that two parameters

In [8]:
import re

def parse_math_prompt(prompt: str) -> str:
    """
    Convert a natural language math prompt into a valid arithmetic expression.
    
    This function applies a series of regex substitutions to handle common
    phrases for operations (multiplication, division, addition, subtraction, power)
    and then cleans up extra words.
    
    For example:
      "I want to multiply 2 and 12 minus 10 power 2"
    becomes:
      "2 * 12 - 10 ** 2"
    
    Args:
        prompt (str): The natural language prompt.
    
    Returns:
        str: A string that is a valid arithmetic expression.
    """
    # Standardize the input to lowercase.
    text = prompt.lower()
    
    # Replace exponentiation phrases first.
    # Handles phrases like "10 power 2" or "10 to the power of 2"
    text = re.sub(r'(\d+(?:\.\d+)?)\s+(?:power|to the power of)\s+(\d+(?:\.\d+)?)', r'\1 ** \2', text)
    
    # Replace multiplication phrases.
    # e.g., "multiply 2 and 12" or "multiplied by"
    text = re.sub(r'multiply\s+(\d+(?:\.\d+)?)(?:\s+and\s+|\s+)(\d+(?:\.\d+)?)', r'\1 * \2', text)
    text = re.sub(r'multiplied\s+by', '*', text)
    
    # Replace division phrases.
    text = re.sub(r'(\d+(?:\.\d+)?)\s+divided\s+by\s+(\d+(?:\.\d+)?)', r'\1 / \2', text)
    text = re.sub(r'divide\s+(\d+(?:\.\d+)?)(?:\s+by\s+)(\d+(?:\.\d+)?)', r'\1 / \2', text)
    
    # Replace addition phrases.
    text = re.sub(r'add\s+(\d+(?:\.\d+)?)(?:\s+and\s+|\s+)(\d+(?:\.\d+)?)', r'\1 + \2', text)
    text = re.sub(r'plus', '+', text)
    
    # Replace subtraction phrases.
    # e.g., "12 minus 10" and also handle "subtract 10 from 12" (which becomes "12 - 10")
    text = re.sub(r'(\d+(?:\.\d+)?)\s+minus\s+(\d+(?:\.\d+)?)', r'\1 - \2', text)
    text = re.sub(r'subtract\s+(\d+(?:\.\d+)?)(?:\s+from\s+)(\d+(?:\.\d+)?)', r'\2 - \1', text)
    
    # Remove common filler words.
    fillers = ["i want to", "what is", "calculate", "please"]
    for filler in fillers:
        text = text.replace(filler, "")
    
    # Remove leftover instances of "and" (which might be used as a conjunction).
    text = text.replace("and", "")
    
    # Clean up extra whitespace.
    expression = " ".join(text.split())
    
    return expression



In [14]:
# Example prompt with more than two operations:
prompt = "I want to multiply 2 and 12 minus 10 power 2"
prompt = "I want to multiply 2 and 5 power 2"
expression = parse_math_prompt(prompt)
print("Extracted Expression:", expression)  # Expected: "2 * 12 - 10 ** 2"

# Evaluate the expression using Python's eval.
result_value = eval(expression)

# Round the result to 2 decimal places if it's not an integer.
if result_value != int(result_value):
    result_value = round(result_value, 2)
else:
    result_value = int(result_value)

print(f"Answer: {result_value}")  # For "2 * 12 - 10 ** 2", expected output: Answer: -76


Extracted Expression: 2 * 5 ** 2
Answer: 50
